In [187]:
import requests
import time
import datetime
import pandas as pd
import numpy as np
from pprint import pprint
import json

In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [3]:
%pylab inline
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

Populating the interactive namespace from numpy and matplotlib


In [4]:
output_notebook()

Loading BokehJS ...

#  API ACCESS TOKEN

In [296]:
import argparse
import ConfigParser
config = ConfigParser.ConfigParser()
config.read('.ttapis.cfg')
config.sections()
ig_config = dict(config.items('Instagram'))

In [295]:
url= 'https://api.instagram.com/oauth/authorize/?client_id=%s&redirect_uri=%s&response_type=token'%(
    ig_config['clientid'],ig_config['redirect_uri'])

print url ### Get access_token

https://api.instagram.com/oauth/authorize/?client_id=8d725859a6da454ca35fa8bdf7a6b3dc&redirect_uri=http://localhost:8123/oauth/access_token&response_type=token


## Username

In [297]:
url = 'https://api.instagram.com/v1/users/self/?access_token=%s'%ig_config['access_token']
requests.get(url).json()

{u'data': {u'bio': u'\u25a0snapchat: @trixandtrax\n\u25b6\ufe0f videos de talentos de tu colegio o universidad',
  u'counts': {u'followed_by': 989, u'follows': 460, u'media': 1529},
  u'full_name': u'TRIX & TRAX',
  u'id': u'227419339',
  u'profile_picture': u'https://scontent.cdninstagram.com/t51.2885-19/s150x150/12534309_1094989960532183_276304202_a.jpg',
  u'username': u'trixandtrax',
  u'website': u'http://www.trixandtrax.com'},
 u'meta': {u'code': 200}}

# RECENT MEDIA

In [298]:
class RecentMedia():
    def __init__(self, userid, access_token):
        self.userid = userid
        self.token = access_token
        pass
    
    def request(self):
        url = 'https://api.instagram.com/v1/users/%s/media/recent/?access_token=%s'%(self.userid,self.token)
        self.recents = requests.get(url).json()
        return self.recents
    
    def build_df(self):
        df = pd.DataFrame(np.nan, index=[0], columns=['empty'])
        insights = []
        insights= [{metric:record[metric] for metric in record.keys()} for record in self.recents['data']]   
        frames = [pd.DataFrame([pd.Series(insight)]) for insight in insights]
        for frame in frames:
            df = pd.concat([df,frame],axis=0)
        self.df = df[1:]
        return self.df
    
    def transform(self):
        self.df['likes']=self.df['likes'].apply(lambda x: x['count'])
        self.df['comments']=self.df['comments'].apply(lambda x: x['count'])
        self.df['created_time']= pd.to_datetime(self.df['created_time'],unit='s')
        return self.df
    
    def combine(self):
        import dill
        with open('df_ig.dill') as f:
            df_ig = dill.load(f)
        self.df_ig = pd.concat([self.df,df_ig], axis=0)
        self.df_ig = self.df_ig.drop_duplicates('id')
        return self.df_ig
    
    def store(self):
        import dill 
        with open('df_ig.dill', 'w') as f:
            dill.dump(self.df_ig,f)
        return self.df_ig

In [322]:
Recmed = RecentMedia(u'227419339',ig_config['access_token'])
Recmed.request()
Recmed.build_df()
Recmed.transform()
df_t = Recmed.combine()
Recmed.store()

,attribution,caption,comments,created_time,empty,filter,id,images,likes,link,location,tags,type,user,user_has_liked,users_in_photo,videos
0,None,"{u'created_time': u'1472267563', u'text': u'Di...",0,2016-08-27 03:12:43,NaN,Walden,1325798872978752884_227419339,{u'low_resolution': {u'url': u'https://sconten...,17,https://www.instagram.com/p/BJmLwzzgwV0/,None,"[4like, like, photo, trixandtrax2016, moment, ...",image,"{u'username': u'trixandtrax', u'profile_pictur...",False,"[{u'position': {u'y': 0.3956383, u'x': 0.60291...",NaN
0,None,"{u'created_time': u'1472176849', u'text': u'En...",1,2016-08-26 02:00:49,NaN,Amaro,1325037906603153516_227419339,{u'low_resolution': {u'url': u'https://sconten...,16,https://www.instagram.com/p/BJjevSpAhhs/,None,"[like, conductor, selfie, girls, trixandtrax20...",image,"{u'username': u'trixandtrax', u'profile_pictur...",True,"[{u'position': {u'y': 0.6453261, u'x': 0.61538...",NaN
0,None,"{u'created_time': u'1472086791', u'text': u'@d...",0,2016-08-25 00:59:51,NaN,Normal,1324282450888791896_227419339,{u'low_resolution': {u'url': u'https://sconten...,12,https://www.instagram.com/p/BJgy99rgpdY/,None,"[good, like, unic, zonabaile, trixandtrax2016,...",video,"{u'username': u'trixandtrax', u'profile_pictur...",True,[],{u'low_resolution': {u'url': u'https://sconten...
0,None,"{u'created_time': u'1471998217', u'text': u'Co...",0,2016-08-24 00:23:37,NaN,Normal,1323539434808920982_227419339,{u'low_resolution': {u'url': u'https://sconten...,14,https://www.instagram.com/p/BJeKBqCAcuW/,None,"[evento, school, like, video, trix, fun, likef...",video,"{u'username': u'trixandtrax', u'profile_pictur...",False,[],{u'low_resolution': {u'url': u'https://sconten...
0,None,"{u'created_time': u'1471997167', u'text': u'Es...",1,2016-08-24 00:06:07,NaN,Normal,1323530630956383522_227419339,{u'low_resolution': {u'url': u'https://sconten...,16,https://www.instagram.com/p/BJeIBizgP0i/,None,"[fun, school, likeforlike, like, freestyle, co...",image,"{u'username': u'trixandtrax', u'profile_pictur...",False,[],NaN
0,None,"{u'created_time': u'1471918091', u'text': u'Aq...",0,2016-08-23 02:08:11,NaN,Normal,1322867291754334897_227419339,{u'low_resolution': {u'url': u'https://sconten...,17,https://www.instagram.com/p/BJbxMsCAAqx/,None,"[like4like, trix, trixandtrax, talentovenezolano]",image,"{u'username': u'trixandtrax', u'profile_pictur...",False,"[{u'position': {u'y': 0.5432998, u'x': 0.59704...",NaN
0,None,"{u'created_time': u'1471911227', u'text': u'Ge...",0,2016-08-23 00:13:47,NaN,Normal,1322809711267020125_227419339,{u'low_resolution': {u'url': u'https://sconten...,13,https://www.instagram.com/p/BJbkGyBgf1d/,None,"[freestyle, iea, football, trix, elpeñón]",video,"{u'username': u'trixandtrax', u'profile_pictur...",False,[],{u'low_resolution': {u'url': u'https://sconten...
0,None,"{u'created_time': u'1471909537', u'text': u'Is...",1,2016-08-22 23:45:37,NaN,Normal,1322795529520018038_227419339,{u'low_resolution': {u'url': u'https://sconten...,22,https://www.instagram.com/p/BJbg4aPg_J2/,None,"[iea, singer, elpeñon, talented, thebest, trax]",video,"{u'username': u'trixandtrax', u'profile_pictur...",True,[],{u'low_resolution': {u'url': u'https://sconten...
0,None,"{u'created_time': u'1471828051', u'text': u'To...",0,2016-08-22 01:07:31,NaN,Normal,1322111980441480119_227419339,{u'low_resolution': {u'url': u'https://sconten...,11,https://www.instagram.com/p/BJZFddjgKu3/,None,"[lomàximo, great, elpeñon, iea, girl, trax]",image,"{u'username': u'trixandtrax', u'profile_pictur...",False,[],NaN
0,None,"{u'created_time': u'1471825789', u'text': u'Wi...",0,2016-08-22 00:29:49,NaN,Normal,1322093003992626843_227419339,{u'low_resolution': {u'url': u'https://sconten...,13,https://www.instagram.com/p/BJZBJUXAnab/,None,"[baile, champagnat]",image,"{u'username': u'trixandtrax', u'profile_pictur...",True,[],NaN


In [323]:
len(df_t)

40

In [303]:
def Ig_Metrics(df, metrics):
    from bokeh.palettes import Spectral11
    p = figure(x_axis_type="datetime", width=900, height=800)
    for i,metric in enumerate(metrics):
        x = pd.to_datetime(df.groupby('created_time')[metric].sum().index)
        y = df.groupby('created_time')[metric].sum().values
        ##x= pd.to_datetime(df.index)
        ##y= df[metric]
        p.line(x,y,color=Spectral11[i*2], legend=metric)


    p.title.text = "Instagram Metrics"
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Metrics'
    p.legend.orientation = "vertical"
    p.legend.location = "top_left"
    return p



In [324]:
show(Ig_Metrics(df_t,['comments','likes']))

# Search by Post ID

In [309]:
url = 'https://api.instagram.com/v1/media/shortcode/%s?access_token=%s'%('BJCSIyBg4GO'# 'BJIIV2ugmF8'# 'BJIIm3_AcCf'
                                                                 ,ig_config['access_token'])
response = requests.get(url).json()


In [310]:
response

{u'data': {u'attribution': None,
  u'caption': {u'created_time': u'1471062946',
   u'from': {u'full_name': u'TRIX & TRAX',
    u'id': u'227419339',
    u'profile_picture': u'https://scontent.cdninstagram.com/t51.2885-19/s150x150/12534309_1094989960532183_276304202_a.jpg',
    u'username': u'trixandtrax'},
   u'id': u'17850463627123040',
   u'text': u'Miguel \xc1ngel con guitarra en mano nos cant\xf3 en la zona #trax #Acapella\xa0#Passenger\xa0@ovamaltina_ve\xa0@Nanivbt115 @_Hebe_ @carmelarowan @Maggysagar  #smile #good #talento #colegio #champagnat #like #4like #moment #guitar #Trixandtrax2016'},
  u'comments': {u'count': 0},
  u'created_time': u'1471062946',
  u'filter': u'Normal',
  u'id': u'1315693809451106702_227419339',
  u'images': {u'low_resolution': {u'height': 320,
    u'url': u'https://scontent.cdninstagram.com/t51.2885-15/s320x320/e15/13628103_318844595124692_861288847_n.jpg?ig_cache_key=MTMxNTY5MzgwOTQ1MTEwNjcwMg%3D%3D.2.l',
    u'width': 320},
   u'standard_resolution': {u

# Search by Tag

In [311]:
url = 'https://api.instagram.com/v1/tags/%s?access_token=%s'%(u'trixandtrax',ig_config['access_token'])
requests.get(url).json()

{u'meta': {u'code': 400,
  u'error_message': u'This request requires scope=public_content, but this access token is not authorized with this scope. The user must re-authorize your application with scope=public_content to be granted this permissions.',
  u'error_type': u'OAuthPermissionsException'}}